In [226]:
import pandas as pd
import numpy as np

In [252]:
ex.tail(7)

,logs,mydate,eid,ename,belongsession,sets,total_reps,total_weight,max_weight,one_RM,category,order,count
3772,"170x10,170x10,170x10",2022-08-25,132,Leg Press,544,3,30,510.0,170.0,226.7,quads,1,29
3773,"60x8,60x8,60x8",2022-08-25,2,Barbell Bench Press,544,3,24,180.0,60.0,76.0,chest,2,149
3774,"40x10,40x10,40x10",2022-08-25,86,Wide Grip Lat Pulldown,544,3,30,120.0,40.0,53.3,back,3,86
3775,"56x10,56x10,56x10",2022-08-25,199,Barbell Romanian Deadlift,544,3,30,168.0,56.0,74.7,hamstrings,4,77
3776,"8x6,8x6,8x6",2022-08-25,24,Dip,544,3,18,24.0,8.0,9.6,triceps,5,70
3777,"140x10,140x10,140x10",2022-08-25,145,Standing Calf Raises,544,3,30,420.0,140.0,186.7,calves,6,114
3778,"9x10,9x10,9x10",2022-08-25,100,Dumbbell Bicep Curl,544,3,30,27.0,9.0,12.0,biceps,7,51


In [251]:
ses.iloc[-1]

date              2022-08-25 00:00:00
time                         16:16:29
total_time                  58.333333
workout_time                     14.7
rest_time                   36.366667
total_exercise                      7
Name: 511, dtype: object

In [227]:
ses = pd.read_csv("sessions_cleaned.csv", index_col=0, parse_dates=["date"])
ex = pd.read_csv("exercises_cleaned.csv", index_col=0, parse_dates=["mydate"])

In [228]:
ex["count"] = ex.groupby("ename").logs.transform("count")
ex2 = ex.loc[ex["count"]>1]
ex2.shape

(3770, 13)

In [229]:
merged = ses.merge(ex2, left_on="date", right_on="mydate", how="inner")
merged["avg_reps"] = merged["total_reps"] / merged["sets"]

In [230]:
pivot = pd.pivot_table(data=merged, index="date", columns="ename", values=["avg_reps", "sets"], fill_value=0)
pivot.shape

(512, 288)

In [231]:
pivot.columns = [(" - ").join([name[1], name[0]]) for name in pivot.columns]

In [232]:
#pivot["total_exercise"] = merged.groupby("date").total_exercise.mean()
X = pivot.reset_index(drop=True)

In [233]:
X

,6'' Block Pull - avg_reps,Air Bike - avg_reps,Arm Circles - avg_reps,Back Hyperextension - avg_reps,Barbell Bench Press - avg_reps,Barbell Bench Squat - avg_reps,Barbell Bent One Arm Row - avg_reps,Barbell Bent-Over Row - avg_reps,Barbell Close Grip Bench Press - avg_reps,Barbell Deadlift - avg_reps,...,Smith Machine Single Leg Split Squat - sets,Standing Calf Raises - sets,T Bar Lying Row - sets,T Bar Prone Row - sets,Thigh Abductor - sets,Touch and Go Deadlift - sets,Weight Plate Decline Crunch - sets,Weight Plate Russian Twist - sets,Weighted Crunches - sets,Wide Grip Lat Pulldown - sets
0,0,0.0,0,0.0,6.0,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,3,0,0,0
1,0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
2,0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,3,0,0,0
3,0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,3,0,0,3
4,0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
508,0,0.0,0,0.0,8.0,0,0.0,0.0,0.0,0.0,...,0,3,0,0,0,0,0,0,0,3
509,0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,5.0,...,0,0,0,3,0,0,0,0,0,0
510,0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [234]:
y = merged.groupby("date")["workout_time"].mean().reset_index(drop=True)
y

0      28.966667
1      19.483333
2      70.016667
3      54.350000
4      21.800000
         ...    
507    16.466667
508    15.016667
509    17.483333
510    12.816667
511    14.700000
Name: workout_time, Length: 512, dtype: float64

In [235]:
from sklearn.svm import SVC
from sklego.meta import ZeroInflatedRegressor

In [236]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score

In [237]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline


In [238]:
from sklearn.metrics import mean_absolute_error
from random import randint

In [239]:
means = []
for random_n in [randint(0,10000) for i in range(100)]:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=random_n)

    clf = Lasso(alpha=0.01, positive=True)
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    clf.fit(X_train_scaled, y_train)

    predictions = clf.predict(X_test_scaled)
    means.append(mean_absolute_error(predictions, y_test))
np.mean(means)

4.076454036915898

In [240]:
coeffs = clf.coef_

In [241]:
coeff_df = pd.DataFrame(columns=X.columns)
coeff_df

,6'' Block Pull - avg_reps,Air Bike - avg_reps,Arm Circles - avg_reps,Back Hyperextension - avg_reps,Barbell Bench Press - avg_reps,Barbell Bench Squat - avg_reps,Barbell Bent One Arm Row - avg_reps,Barbell Bent-Over Row - avg_reps,Barbell Close Grip Bench Press - avg_reps,Barbell Deadlift - avg_reps,...,Smith Machine Single Leg Split Squat - sets,Standing Calf Raises - sets,T Bar Lying Row - sets,T Bar Prone Row - sets,Thigh Abductor - sets,Touch and Go Deadlift - sets,Weight Plate Decline Crunch - sets,Weight Plate Russian Twist - sets,Weighted Crunches - sets,Wide Grip Lat Pulldown - sets


In [242]:
coeff_df.loc[0] = coeffs

In [243]:
half = int((len(coeff_df.columns)-1)/2)

In [244]:
reps = coeff_df.iloc[:,:half].transpose()

In [245]:
reps[reps[0]>0].sort_values(by=0, ascending=False)

,0
Smith Machine Single Leg Split Squat - avg_reps,16.866171
Cable One Arm Lateral Raise - avg_reps,10.018020
Flutter Kick - avg_reps,9.857125
Cable Reverse Wood Chops - avg_reps,9.412896
Dumbbell Side Bend - avg_reps,8.743736
Dumbbell Standing One Arm Triceps Extension - avg_reps,8.174643
Dumbbell Shoulder Shrug - avg_reps,8.171364
Dumbbell Lunges - avg_reps,7.557630
Cable One Arm Standing Overhead Tricep Extension - avg_reps,5.722324
Calf Press On Leg Press - avg_reps,5.547784


In [246]:
sets = coeff_df.iloc[:,half:].transpose()

In [247]:
sets[sets[0]>0].sort_values(by=0, ascending=False)

,0
Weight Plate Decline Crunch - sets,7.827916
Pull-Up - sets,6.834526
Cross Body Crunch - sets,6.765796
Hanging Knee Raise - sets,6.394639
Dumbbell Walking Lunges - sets,6.258280
Elliptical Training - sets,5.620154
Dumbbell Bent Over Reverse Fly - sets,5.405057
Dumbbell Incline Bench Two Arm Row - sets,5.074958
Dumbbell Incline Bench Press - sets,4.972061
Dumbbell Palms In Bench Press - sets,4.753853


In [248]:
clf.intercept_

4.709066160100754

In [249]:
last = np.asarray(X_scaled_df.iloc[450]).reshape(1,-1)
clf.predict(last)

NameError: name 'X_scaled_df' is not defined

In [ ]:
y.iloc[450]

61.54999999999999

In [ ]:
coeffs

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  1.00449359,  0.        ,  1.70741711,
       -0.        , -0.        ,  1.64312037, -0.02011761,  0.        ,
       -0.        , -0.10219509,  0.        ,  0.        , -0.30836455,
       -0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
       -0.        , -0.        ,  0.        , -0.        ,  0.        ,
       -0.        ,  0.72288113,  0.        , -0.        ,  2.94433419,
       -0.56231409,  0.1606803 ,  0.        , -0.        ,  2.39093184,
        1.88694004,  1.45930007,  0.65926467,  0.79345968,  0.59861794,
        2.45795686,  0.        ,  0.        ,  1.07024781,  0.        ,
       -0.        ,  0.        , -0.        ,  0.        , -0.        ,
       -0.        ,  1.15819635,  0.        , -0.        , -0.        ,
        0.        ,  0.        ,  0.        ,  0.1480541 ,  0.        ,
       -0.        ,  0.10154145,  0.        ,  0.        , -0.  

In [ ]:
ses.head(10)

,date,time,total_time,total_exercise
0,2019-03-08,17:00:57,66.816667,9
1,2019-03-09,16:29:21,62.483333,12
2,2019-03-10,18:34:35,117.066667,11
3,2019-03-12,18:09:51,103.350000,15
4,2019-03-13,21:14:38,52.016667,7
5,2019-03-14,16:21:51,100.016667,11
6,2019-03-19,14:38:09,96.850000,11
7,2019-03-20,18:27:54,60.716667,10
8,2019-03-21,18:40:00,95.883333,11
9,2019-03-22,16:41:51,62.916667,7
